In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

from mis_utils import *

In [2]:
# Definiciones de constantes

node_names = np.array(('1', '2', '3', 'a', 'b'))

# Balances: Sale un producto desde s y llega hasta t
beq = np.array([10, 20, 15, -25, -20])

NN = np.array([[0, 0, 0, 1, 1],
               [0, 0, 0, 1, 1],
               [0, 0, 0, 1, 1],
               [0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0]])

# Matrices resultantes de NN a NA
Aeq, arc_idxs = nn2na(NN, node_names = node_names, show_results = False)

# Guardo los nombres de los nodo-arco-nodo posibles
nan_names = get_col_names(NN, node_names, as_numpy=True, sep = "->")

# Restricciones, l <= x <= u
# Entre 0 y 1 sería lo teóricamente correcto, aunque como hay un límite que como el modelo indica, es de un producto
min_bound = 0
# max_bound = np.array([10,20,10,10,10,30])
max_bound = None
bounds = tuple([(min_bound, None) for arcs in range(0, Aeq.shape[1])])
# bounds = tuple([(min_bound, arcs) for arcs in max_bound])

In [3]:
pd.DataFrame(data=NN, columns=node_names, index = node_names)

,1,2,3,a,b
1,0,0,0,1,1
2,0,0,0,1,1
3,0,0,0,1,1
a,0,0,0,0,0
b,0,0,0,0,0


In [4]:
pd.DataFrame(data=Aeq, columns=nan_names, index = node_names)

,1->a,1->b,2->a,2->b,3->a,3->b
1,1,1,0,0,0,0
2,0,0,1,1,0,0
3,0,0,0,0,1,1
a,-1,0,-1,0,-1,0
b,0,-1,0,-1,0,-1


In [5]:
# Vector de costos
# Con el orden de los nombres de columnas, sacado de los resultados. Column names: s2-s3-24-2t-35-4t-5t
C = np.array([10, 20, 10, 10, 10, 30])
# C = np.zeros(Aeq.shape[1]) # Mal, me brindan los costos
# C = np.ones(Aeq.shape[1]) # Mal, me bindan los costos

In [6]:
df_aux = pd.DataFrame(C)

In [7]:
df_info = pd.DataFrame(bounds, index=nan_names, columns=['Bound min', 'Bound max'])
df_info['Cost'] = C
df_info

,Bound min,Bound max,Cost
1->a,0,None,10
1->b,0,None,20
2->a,0,None,10
2->b,0,None,10
3->a,0,None,10
3->b,0,None,30


In [8]:
# Resumen
print('## Optimizer inputs ## \n\n'
      'Cost vector: %s \n\n'
      'Columns: %s \n\n'
      'A_eq Node-Arc matrix:\n%s \n\n'
      'b_eq demand-supply vector: %s \n\n'
      'Bounds of each X arc variable: %s \n' % (C, nan_names, Aeq, beq, bounds))

## Optimizer inputs ## 

Cost vector: [10 20 10 10 10 30] 

Columns: ['1->a' '1->b' '2->a' '2->b' '3->a' '3->b'] 

A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0]
 [ 0  0  1  1  0  0]
 [ 0  0  0  0  1  1]
 [-1  0 -1  0 -1  0]
 [ 0 -1  0 -1  0 -1]] 

b_eq demand-supply vector: [ 10  20  15 -25 -20] 

Bounds of each X arc variable: ((0, None), (0, None), (0, None), (0, None), (0, None), (0, None)) 



# Resuelvo con el método simplex

In [9]:
# Optimización
res_simplex = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')

selarcs = get_selected_arcs(res_simplex.x, nan_names)

/home/diego/.virtualenvs/OS11/lib/python3.6/site-packages/ipykernel_launcher.py:2: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  


In [10]:
print('## Results ##')
print('The raw solution will be: %s' % res_simplex.x)
print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
print('The minimum cost will be: %0.2f ' % res_simplex.fun)

## Results ##
The raw solution will be: [10.  0.  0. 20. 15.  0.]
The arcs that make the shortest path will be (from, to): []
The minimum cost will be: 450.00 


# Recordatorio: El método simplex no resuelve problemas enteros pero se relaja el problema para tal método, entender mejor cómo opera eso desde lo teórico.
## Es una aproximación o tendría que hacer algo para limitarlo?

In [11]:
df_info['Solución'] = res_simplex.x.round().astype(int)

In [12]:
df_info

,Bound min,Bound max,Cost,Solución
1->a,0,None,10,10
1->b,0,None,20,0
2->a,0,None,10,0
2->b,0,None,10,20
3->a,0,None,10,15
3->b,0,None,30,0
